In [1]:
import pandas as pd
import plotly as pt
import folium
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [57]:
confirmed_cases = pd.read_csv('Data/time_series_covid19_confirmed_global.csv')

country_coordinates = {}

for index,row in confirmed_cases.iterrows():
    if row['Country/Region'] in country_coordinates:
        pass
    else:
        country_coordinates[row['Country/Region']] = (row.Lat, row.Long)

country_coordinates

{'Afghanistan': (33.93911, 67.709953),
 'Albania': (41.1533, 20.1683),
 'Algeria': (28.0339, 1.6596),
 'Andorra': (42.5063, 1.5218),
 'Angola': (-11.2027, 17.8739),
 'Antarctica': (-71.9499, 23.347),
 'Antigua and Barbuda': (17.0608, -61.7964),
 'Argentina': (-38.4161, -63.6167),
 'Armenia': (40.0691, 45.0382),
 'Australia': (-35.4735, 149.0124),
 'Austria': (47.5162, 14.5501),
 'Azerbaijan': (40.1431, 47.5769),
 'Bahamas': (25.025885, -78.035889),
 'Bahrain': (26.0275, 50.55),
 'Bangladesh': (23.685, 90.3563),
 'Barbados': (13.1939, -59.5432),
 'Belarus': (53.7098, 27.9534),
 'Belgium': (50.8333, 4.469936),
 'Belize': (17.1899, -88.4976),
 'Benin': (9.3077, 2.3158),
 'Bhutan': (27.5142, 90.4336),
 'Bolivia': (-16.2902, -63.5887),
 'Bosnia and Herzegovina': (43.9159, 17.6791),
 'Botswana': (-22.3285, 24.6849),
 'Brazil': (-14.235, -51.9253),
 'Brunei': (4.5353, 114.7277),
 'Bulgaria': (42.7339, 25.4858),
 'Burkina Faso': (12.2383, -1.5616),
 'Burma': (21.9162, 95.956),
 'Burundi': (-3.

In [3]:
confirmed_cases_by_country = confirmed_cases.groupby("Country/Region").sum(numeric_only=True)\
                                            .drop(columns=['Lat', 'Long'])

In [4]:
def date_converter(date_str):
    month_map = {'1': "January", '2': "February", '3': "March", '4': "April", '5': "May", '6': "June", 
                 '7': "July", '8': "August", '9': "September", '10': "October", '11': "November", '12': "December"}

    split_date = date_str.split('/')

    return month_map[split_date[0]] + split_date[-1]

In [5]:
month_list = []
for i in confirmed_cases_by_country.columns:
    month_list.append(date_converter(i))

confirmed_cases_by_country.columns = month_list

In [ ]:
monthly_data_df = pd.DataFrame()

for month in pd.unique(confirmed_cases_by_country.columns):
    x = confirmed_cases_by_country[month]
    monthly_data_df[month] = x.max(axis=1)

# monthly_data_df.to_csv('clean_data.csv')

In [ ]:
globe = folium.Map(location=[54.5, 15.25], zoom_start=2)

final_df = monthly_data_df.max(axis=1).to_frame(name='max_cases')
final_df['ranked'] = (np.log(final_df['max_cases']).round(2))/5

for country in country_coordinates:
    country_name = country
    ranked_score = final_df.loc[country_name, 'max_cases']

    html = f"""
    <h4 style="margin-bottom: 5px;">Country:</h4>
    <p><strong>{country_name}</strong></p>

    <h4 style="margin-bottom: 5px;">Total Cumulative Cases:</h4>
    <p><strong>{ranked_score:.2f}</strong></p>
    """
    
    folium.CircleMarker(
        location=country_coordinates[country],
        radius=final_df.loc[country, 'ranked'],
        fill=True,
        popup=folium.Popup(html, max_width=300)
    ).add_to(globe)

display(globe)

In [53]:
def month_selector(month):
    """
    This function streamlines the full dataset to the chosen month.
    """
    globe = folium.Map(location=[54.5, 15.25], zoom_start=2)

    df = monthly_data_df.loc[:, :month]\
                        .max(axis=1)\
                        .to_frame(name='max_cases')

    df['ranked'] = (np.log(df['max_cases']).round(2))/5

    for country in country_coordinates:
        country_name = country
        ranked_score = df.loc[country_name, 'max_cases']

        html = f"""
        <h4 style="margin-bottom: 5px;">Country:</h4>
        <p><strong>{country_name}</strong></p>

        <h4 style="margin-bottom: 5px;">Total Cumulative Cases:</h4> 
        <p><strong>{ranked_score:.2f}</strong></p>
        """
    
        folium.CircleMarker(
        location=country_coordinates[country],
        radius=df.loc[country, 'ranked'],
        fill=True,
        popup=folium.Popup(html, max_width=300)
        ).add_to(globe)

    display(globe)

In [ ]:
month_selector('September20')